In [ ]:
# %tensorflow_version 1.x


In [ ]:
#install dependencies
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

#install magenta
!pip install -qU magenta

import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files

import magenta
import note_seq
import tensorflow


Create a notesequence for *Twinkle Twinkle Little Star*

In [ ]:
from note_seq.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()

twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# visualization
note_seq.plot_sequence(twinkle_twinkle)

# play
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

In [ ]:
# output midi file
note_seq.sequence_proto_to_midi_file(twinkle_twinkle, 'twinkle_twinkle_output.mid')

# download the file
#files.download('twinkle_twinkle_output.mid')

Extend Twinkle Twinkle Little Star with Melody RNN

In [ ]:
# !wget http://download.magenta.tensorflow.org/models/basic_rnn.mag

--2020-12-01 02:05:23--  http://download.magenta.tensorflow.org/models/basic_rnn.mag
Resolving download.magenta.tensorflow.org (download.magenta.tensorflow.org)... 173.194.216.128, 2607:f8b0:400c:c12::80
Connecting to download.magenta.tensorflow.org (download.magenta.tensorflow.org)|173.194.216.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13044885 (12M) [binary/octet-stream]
Saving to: ‘basic_rnn.mag.2’

basic_rnn.mag.2     100%[===================>]  12.44M  70.9MB/s    in 0.2s    

2020-12-01 02:05:24 (70.9 MB/s) - ‘basic_rnn.mag.2’ saved [13044885/13044885]



In [ ]:
# download the pretrained basic_rnn model
# note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')

from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

# initialized melody RNN
bundle = sequence_generator_bundle.read_bundle_file('basic_rnn (1).mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpds48pjry/model.ckpt


In [ ]:
input_sequence = twinkle_twinkle 
num_steps = 128
temperature = 1 # less than 1, less random

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)


INFO:tensorflow:Beam search yields sequence with log-likelihood: -105.876358 


Extend Twinkle Twinkle Little Star with RL Tuner

In [ ]:
from magenta.contrib import training as contrib
basic_rnn_hparams=contrib.HParams(batch_size=128,rnn_layer_sizes=[128,128],dropout_keep_prob=0.5,clip_norm=5,learning_rate=0.001, one_hot_length=8)

In [ ]:
from magenta.models.rl_tuner import rl_tuner
from magenta.models.rl_tuner import rl_tuner_ops
from magenta.models.rl_tuner import note_rnn_loader
from magenta.models.rl_tuner import rl_tuner_eval_metrics

rl_net = rl_tuner.RLTuner(output_dir='/content',
                          priming_mode='single_midi',
                          midi_primer='/content/twinkle_twinkle_output.mid',
                          note_rnn_checkpoint_dir='/content/',
                          note_rnn_type='basic_rnn',
                          note_rnn_hparams=basic_rnn_hparams,
                          num_notes_in_melody=128,
                          save_name='rl_tuner.ckpt')